#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

In [9]:
import pandas as pd
from sentence_transformers import SentenceTransformer

In [10]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [11]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [30]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)
vectors

array([[-0.00247396,  0.03626723, -0.05290459, ..., -0.09152357,
        -0.03970001, -0.04330489],
       [-0.03357268,  0.00980517, -0.03250129, ..., -0.05165466,
         0.02245888, -0.03156181],
       [-0.01865322, -0.04051315, -0.01235386, ...,  0.00610586,
        -0.07179644,  0.02773852],
       ...,
       [-0.00066459,  0.04252128, -0.05645508, ...,  0.01315471,
        -0.03183568, -0.04357662],
       [-0.03317154,  0.03252454, -0.02484838, ...,  0.0117442 ,
         0.05747125,  0.00571021],
       [-0.00166395,  0.00413827, -0.04597083, ...,  0.02008528,
         0.05656242, -0.00161596]], dtype=float32)

In [13]:
vectors.shape

(8, 768)

In [14]:
dim = vectors.shape[1]
dim

768

###  Build a FAISS Index for vectors

In [33]:
import faiss

index = faiss.IndexFlatL2(dim) ## eucledian distance crete 768 indexes to add the vectors

In [34]:
index.add(vectors) 

In [37]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001A37CC09BF0> >

### Encode search text using same encorder and normalize the output vector

In [18]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [28]:
type(vec) # as the type of vec id array , we have to convert it into a 2d array search takes a2d array

numpy.ndarray

In [39]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [22]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844835, 1.4039094]], dtype=float32)

In [23]:
I.tolist()

[[3, 2]]

In [24]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [25]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [26]:
search_query

'I want to buy a polo t-shirt'